In [8]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt

In [7]:
REVIEWS_FILE = "./data/Grocery_and_Gourmet_Food.json"

In [17]:
def load_pickle(pickle_path):
    with open(pickle_path, 'rb') as file:
        return pickle.load(file)

In [21]:
def save_pickle(result, pickle_path):
    with open(pickle_path, 'wb') as file:
        pickle.dump(result, file)

In [22]:
def read_reviews(path=REVIEWS_FILE, pickle_path="./temp/reviews"):
    try:
        return load_pickle(pickle_path)
    except (FileNotFoundError, EOFError) as e:
        reviews = pd.read_json(REVIEWS_FILE, lines=True)
        save_pickle(reviews, pickle_path)
        return reviews

In [23]:
reviews = read_reviews()

OSError: [Errno 122] Disk quota exceeded

In [ ]:
reviews